In [5]:
import time
import emoji
import re
import random
import requests
import pandas as pd
from bs4 import BeautifulSoup

def random_headers():
    headers_list = [{'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0'},{'User-agent':' Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'},{'User-agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36 OPR/38.0.2220.41'},{'User-agent':'Mozilla/5.0 (Linux; U; Android 4.0.3; de-ch; HTC Sensation Build/IML74K) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 Mobile Safari/534.30'},{'User-agent':'Mozilla/5.0 (compatible; MSIE 9.0; Windows Phone OS 7.5; Trident/5.0; IEMobile/9.0)'}]
    headers=random.choice(headers_list)
    #print(headers)
    return headers
    

  
def get_soup(url,headers):
    ss=requests.session()
    res=ss.get(url,headers=headers)
    
    res_status_code=res.status_code
    print(res_status_code)
    
    if res_status_code!=200:
        proxy_list=[{'https':'59.124.224.180:3128','http':'59.124.224.180:3128'},{'https':'113.196.140.162:8888','http':'113.196.140.162:8888'}]
        proxy_list2={'http':'http://220.135.165.38:8080','https':'220.135.168.38:8080'}
        proxy=random.choice(proxy_list)
        
        print("進入proxy狀態,ip已經被鎖定")
        try:
            time.sleep(5)
            
            res = requests.get(url ,proxies=proxy ,headers=headers)
            res_status_code=res.status_code
            print(proxy)
            print(res_status_code)
            #print(res.status_code)
            
            soup=BeautifulSoup(res.text,'html.parser')
            return soup
            
            #print(res.json())    
   
        except Exception as e:
            proxy=proxy_list2
            time.sleep(5)
            res = requests.get(url ,proxies=proxy ,headers=headers)
            res_status_code=res.status_code
            print(res_status_code)
           
            print("這是備源proxy")
            res_status_code=res.status_code
            if res_status_code!=200:
                print("所有proxy失效,程式終止")
                
            else:
              
                soup=BeautifulSoup(res.text,'html.parser')
                return soup
           
    
    else:
        
       
        #print(ss.cookies.get_dict())

        #res=ss.get(url,headers=headers)
        soup=BeautifulSoup(res.text,'html.parser')
        return soup

def preprocess (x):
    x = emoji.get_emoji_regexp().sub(r'', x)
    x = re.sub(r'[［(（【{][^】)}）］]*[】)}）］]', '', x)
    x = x.strip().replace(" ", "")
    x = re.sub(r'[~\-。]', "", x) 
    return x

headers = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'}
domain = 'https://icook.tw/'
url = 'categories/455'

c = 0
url = domain + url
df = pd.DataFrame(columns = ['Recipe Name','Author','Recipe URL','Recipe Picture','Servings','Time','Ingredients',\
                             'Unit','Recipe Like','Publish Date','Views Number','Recipe Steps'])

headers=random_headers()
soup=get_soup(url,headers)
vip_count=0
page = 1

time_start = time.time()



total_amount = soup.select('title')[0].text
total_amount = int(''.join(re.findall("\d+", total_amount)))
max_page = 0
if total_amount % 18 == 0:
    max_page = total_amount // 18
else:
    max_page = (total_amount // 18) + 1



while(page <= max_page):
    page_index = f'?page={page}'
    url = domain + url + page_index
    print(f"URL: {url}")
    
    headers=random_headers()
    soup=get_soup(url,headers)
    food_list = soup.select('a.browse-recipe-touch-link')
    for food in food_list:
        url = 'https://icook.tw/' + food['href']
         headers=random_headers()
        soup=get_soup(url,headers)
        print(url)
        #recipe_title = "".join((soup_recipe.select('h1.title')[0].text).split())
        #recipe_title = preprocess(recipe_title)
        recipe_title = soup.select('h1.title')[0].text
        recipe_title = preprocess(recipe_title)
        check_vip=soup.select('h1.title')[0].select('span.badge-vip')
        if len(check_vip)<=0:
            pass
            #print("is not vip data, data will be save")
        else:
            print("this is vip data,data will not save")
            vip_count+=1
            continue
        recipe_author = soup.select('a.author-name-link')[0].text
        recipe_pic_url = soup.select("img.main-pic")[0]['src']
        
    
        if len(soup.select('span.stat-content')) == 2:
            recipe_like = 0
        else:
            if '萬' in soup.select('span.stat-content')[0].text:
                recipe_like = int(soup.select('span.stat-content')[0].text[:-5].replace('.','')+"000")
            else:
                recipe_like = int(soup.select('span.stat-content')[0].text[:-3].replace(',',''))
        
        if not soup.select('div.servings-info.info-block') and not soup.select('div.time-info.info-block'):
            recipe_servings = 0
            recipe_time = 0
        elif not soup.select('div.time-info.info-block'):
            recipe_servings = int(soup.select('span.num')[0].text)
            recipe_time = 0
        elif  not soup.select('div.servings-info.info-block'): 
            recipe_servings = 0
            recipe_time = int(soup.select('span.num')[0].text)
        else: 
            recipe_servings = int(soup.select('span.num')[0].text)
            recipe_time = int(soup.select('span.num')[1].text)

        recipe_ingredient = ""
        for ingredient in soup.select('.ingredient-search'):
            if ingredient.text == "請參考圖片說明":
                continue
            else:
                recipe_ingredient += ingredient.text + " " 
        recipe_ingredient = emoji.get_emoji_regexp().sub(r'', recipe_ingredient)
        
        recipe_unit = ""
        for unit in soup.select('div.ingredient-unit'):
            if unit.text == "内文有":
                continue
            else:
                recipe_unit += unit.text + " "

        recipe_steps = ""
        counter = 1
        for steps in soup.select('div.step-instruction-content'):
            recipe_steps += str(counter) + " " + steps.text + "\n"
            recipe_steps = emoji.get_emoji_regexp().sub(r'', recipe_steps)
            counter += 1
            
        publish_date = soup.select('span.meta-content')
        if len (publish_date)<=0:
            print("publish_date is na")
            publish_date="na"
        else:
            publish_date = publish_date[0].text[:-3]
        
        views = soup_.select('span.meta-content')
        if len(views)<=0:
            print("view is na")
            views_number=0
            views="0"
        else:
            views=views[1].text[:-3]
            if '萬' in views:
                views_number = views[:-2]
                views_number = int(views_number.replace('.','')+"000")
            else:
                views_number = int(views.replace(',',''))
        
        
        
        #print(recipe_section)    
        #print(recipe_url)
        #print(recipe_title)
        #print(recipe_author)
        #print(recipe_pic_url)
        
        print(recipe_like)
        #print(recipe_servings)
        #print(recipe_time)
        #print(recipe_ingredient)
        #print(recipe_unit)
        #print(recipe_steps)
        print(publish_date)
        #print(views)
        print("----")
        
        ls = [recipe_title,recipe_author,recipe_url,recipe_pic_url,recipe_servings,recipe_time,recipe_ingredient,\
              recipe_unit,recipe_like,publish_date,views_number,recipe_steps]
        df.loc[c] = ls
        c += 1
    
        time.sleep(random.randint(1,4))
    page += 1        
df
print('有多少vip資料{}'.format(vip_count))
time_end = time.time()
print(time_end - time_start)


200


'\nwhile(page <= max_page):\n    page_index = f\'?page={page}\'\n    url = domain + url + page_index\n    print(f"URL: {url}")\n    random_headers()\n    headers=headers\n    get_soup(url,headers)\n    soup=get_soup(url,headers)\n    food_list = soup.select(\'a.browse-recipe-touch-link\')\n    for food in food_list:\n        url = \'https://icook.tw/\' + food[\'href\']\n         \n        random_headers()\n        headers=headers\n        get_soup(url,headers)\n        soup=get_soup(url,headers)\n        print(url)\n        #recipe_title = "".join((soup_recipe.select(\'h1.title\')[0].text).split())\n        #recipe_title = preprocess(recipe_title)\n        recipe_title = soup.select(\'h1.title\')[0].text\n        recipe_title = preprocess(recipe_title)\n        check_vip=soup.select(\'h1.title\')[0].select(\'span.badge-vip\')\n        if len(check_vip)<=0:\n            pass\n            #print("is not vip data, data will be save")\n        else:\n            print("this is vip data,dat

In [ ]:
df

In [ ]:
df.to_csv(f'./{section_number}.csv', index=False ,encoding='utf-8-sig')

In [ ]:
domain = 'https://icook.tw/'
url = 'categories/455'

c = 0
url = domain + url


ss=requests.session()
res=ss.get(url,headers=headers)
    
res_status_code=res.status_code
print(res_status_code)